# Geometry operations {#geometric-operations}

## Prerequisites


In [ ]:
#| echo: false
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.max_rows = 6
pd.options.display.max_columns = 6
pd.options.display.max_colwidth = 35
plt.rcParams['figure.figsize'] = (5, 5)

Packages...


In [ ]:
import numpy as np
import shapely.geometry
import geopandas as gpd
import topojson as tp
import rasterio
import rasterio.warp
import rasterio.plot

Sample data...


In [ ]:
seine = gpd.read_file('data/seine.gpkg')
us_states = gpd.read_file('data/us_states.gpkg')
nz = gpd.read_file('data/nz.gpkg')

## Introduction

So far the book has explained the structure of geographic datasets (Chapter 2), and how to manipulate them based on their non-geographic attributes (Chapter 3) and spatial relations (Chapter 4). This chapter focusses on manipulating the geographic elements of geographic objects, for example by simplifying and converting vector geometries, cropping raster datasets, and converting vector objects into rasters and from rasters into vectors. After reading it---and attempting the exercises at the end---you should understand and have control over the geometry column in sf objects and the extent and geographic location of pixels represented in rasters in relation to other geographic objects.

@sec-geo-vec covers transforming vector geometries with 'unary' and 'binary' operations. Unary operations work on a single geometry in isolation, including simplification (of lines and polygons), the creation of buffers and centroids, and shifting/scaling/rotating single geometries using 'affine transformations' (@sec-simplification to @sec-affine-transformations). Binary transformations modify one geometry based on the shape of another, including clipping and geometry unions, covered in @sec-clipping and @sec-geometry-unions, respectively. Type transformations (from a polygon to a line, for example) are demonstrated in Section @sec-type-transformations.

@sec-geo-ras covers geometric transformations on raster objects. This involves changing the size and number of the underlying pixels, and assigning them new values. It teaches how to change the resolution (also called raster aggregation and disaggregation), the extent and the origin of a raster. These operations are especially useful if one would like to align raster datasets from diverse sources. Aligned raster objects share a one-to-one correspondence between pixels, allowing them to be processed using map algebra operations, described in Section 4.3.2. The final Section 6 connects vector and raster objects. It shows how raster values can be 'masked' and 'extracted' by vector geometries. Importantly it shows how to 'polygonize' rasters and 'rasterize' vector datasets, making the two data models more interchangeable.

## Geometric operations on vector data {#sec-geo-vec}

This section is about operations that in some way change the geometry of vector layers. It is more advanced than the spatial data operations presented in the previous chapter (in @sec-spatial-vec), because here we drill down into the geometry: the functions discussed in this section work on the geometric (`GeoSeries`) part, either as standalone object or as part of a `GeoDataFrame`.

### Simplification {#sec-simplification}

Simplification is a process for generalization of vector objects (lines and polygons) usually for use in smaller scale maps. Another reason for simplifying objects is to reduce the amount of memory, disk space and network bandwidth they consume: it may be wise to simplify complex geometries before publishing them as interactive maps. The `geopandas` package provides the `.simplify` method, which uses the GEOS implementation of the Douglas-Peucker algorithm to reduce the vertex count. `.simplify` uses the `tolerance` to control the level of generalization in map units (see Douglas and Peucker 1973 for details). 

For example, a simplified geometry of a `"LineString"` geometry, representing the river Seine and tributaries, using tolerance of `2000` meters, can created using the following command:


In [ ]:
seine_simp = seine.simplify(2000)  # 2000 m

Figure @fig-simplify-lines illustrates the input and the result of the simplification:


In [ ]:
#| label: fig-simplify-lines
#| fig-cap: Comparison of the original and simplified geometry of the seine object.

fig, axes = plt.subplots(ncols=2)
seine.plot(ax=axes[0])
seine_simp.plot(ax=axes[1])
axes[0].set_title('Original')
axes[1].set_title('Simplified (d=2000 m)');

The resulting `seine_simp` object is a copy of the original `seine` but with fewer vertices. This is apparent, with the result being visually simpler (@fig-simplify-lines, right) and consuming less memory than the original object, as verified below:


In [ ]:
import sys
sys.getsizeof(seine)       ## Original (bytes)

In [ ]:
sys.getsizeof(seine_simp)  ## Simplified (bytes)

Simplification is also applicable for polygons. This is illustrated using `us_states`, representing the contiguous United States. As we show in @sec-reproj-geo-data, GEOS assumes that the data is in a projected CRS and this could lead to unexpected results when using a geographic CRS. Therefore, the first step is to project the data into some adequate projected CRS, such as US National Atlas Equal Area (epsg = `2163`) (on the left in Figure @fig-simplify-polygons):


In [ ]:
us_states2163 = us_states.to_crs(2163)

The `.simplify` method from `geopandas` works the same way with a `"Polygon"`/`"MultiPolygon"` layer such as `us_states2163`:


In [ ]:
us_states_simp1 = us_states2163.simplify(100000)

A limitation with `.simplify` is that it simplifies objects on a per-geometry basis. This means the "topology" is lost, resulting in overlapping and "holey" areal units illustrated in Figure @fig-simplify-polygons (middle panel). The `toposimplify` function from `topojson` provides an alternative that overcomes this issue. By [default](https://mattijn.github.io/topojson/example/settings-tuning.html#simplify_algorithm) it uses the Douglas-Peucker algorithm like the `.simplify` method. Another algorithm known as Visvalingam-Whyatt, which overcomes some limitations of the Douglas-Peucker algorithm (Visvalingam and Whyatt 1993), is also available in `toposimplify`. The main advanatage of `toposimplify`, however, is that it is topologically "aware". That is, it simplifies the combined borders of the polygons (rather than each polygon on its own), thus ensuring that the overlap is maintained. The following code chunk uses this function to simplify `us_states2163`:


In [ ]:
topo = tp.Topology(us_states2163, prequantize=False)
us_states_simp2 = topo.toposimplify(100000).to_gdf()

Figure @fig-simplify-polygons demonstrates the two simplification methods applied to `us_states2163`.


In [ ]:
#| label: fig-simplify-polygons
#| fig-cap: 'Polygon simplification in action, comparing the original geometry of the contiguous United States with simplified versions, generated with functions from the geopandas (middle), and topojson (right), packages.'

fig, axes = plt.subplots(ncols=3, figsize=(8,4))
us_states2163.plot(ax=axes[0], color='lightgrey', edgecolor='black')
us_states_simp1.plot(ax=axes[1], color='lightgrey', edgecolor='black')
us_states_simp2.plot(ax=axes[2], color='lightgrey', edgecolor='black')
axes[0].set_title("Original")
axes[1].set_title("Simplified (w/ geopandas)")
axes[2].set_title("Simplified (w/ topojson)");

### Centroids

Centroid operations identify the center of geographic objects. Like statistical measures of central tendency (including mean and median definitions of 'average'), there are many ways to define the geographic center of an object. All of them create single point representations of more complex vector objects.

The most commonly used centroid operation is the geographic centroid. This type of centroid operation (often referred to as 'the centroid') represents the center of mass in a spatial object (think of balancing a plate on your finger). Geographic centroids have many uses, for example to create a simple point representation of complex geometries, or to estimate distances between polygons. Centroids of the geometries in a `GeoSeries` or a `GeoDataFrame` are accessible through the `.centroid` property, as demonstrated in the code below, which generates the geographic centroids of regions in New Zealand and tributaries to the River Seine, illustrated with black points in @fig-centroid-pnt-on-surface.


In [ ]:
nz_centroid = nz.centroid
seine_centroid = seine.centroid

Sometimes the geographic centroid falls outside the boundaries of their parent objects (think of a doughnut). In such cases point on surface operations can be used to guarantee the point will be in the parent object (e.g., for labeling irregular multipolygon objects such as island states), as illustrated by the red points in @fig-centroid-pnt-on-surface. Notice that these red points always lie on their parent objects. They were created with the `representative_point` method, as follows:


In [ ]:
nz_pos = nz.representative_point()
seine_pos = seine.representative_point()

The centroids and points in surface are illustrated in @fig-centroid-pnt-on-surface:


In [ ]:
#| label: fig-centroid-pnt-on-surface
#| fig-cap: Centroids (black) and points on surface red of New Zealand and Seine datasets.

fig, axes = plt.subplots(ncols=2)
nz.plot(ax=axes[0], color="white", edgecolor="lightgrey")
nz_centroid.plot(ax=axes[0], color="None", edgecolor="black")
nz_pos.plot(ax=axes[0], color="None", edgecolor="red")
seine.plot(ax=axes[1], color="grey")
seine_pos.plot(ax=axes[1], color="None", edgecolor="red")
seine_centroid.plot(ax=axes[1], color="None", edgecolor="black");

### Buffers {#sec-buffers}

Buffers are polygons representing the area within a given distance of a geometric feature: regardless of whether the input is a point, line or polygon, the output is a polygon. Unlike simplification (which is often used for visualization and reducing file size) buffering tends to be used for geographic data analysis. How many points are within a given distance of this line? Which demographic groups are within travel distance of this new shop? These kinds of questions can be answered and visualized by creating buffers around the geographic entities of interest.

@fig-buffers illustrates buffers of different sizes (5 and 50 km) surrounding the river Seine and tributaries. These buffers were created with commands below, which show that the `.buffer` method, applied to a `GeoSeries` (or `GeoDataFrame`) requires one important argument: the buffer distance, provided in the units of the CRS (in this case meters):


In [ ]:
seine_buff_5km = seine.buffer(5000)
seine_buff_50km = seine.buffer(50000)

The 5 and 50 km buffers are visualized in @fig-buffers:


In [ ]:
#| label: fig-buffers
#| fig-cap: 'Buffers around the Seine dataset of 5 km (left) and 50 km (right). Note the colors, which reflect the fact that one buffer is created per geometry feature.'

fig, axes = plt.subplots(ncols=2)
seine_buff_5km.plot(ax=axes[0], color="none", edgecolor=["red", "green", "blue"])
seine_buff_50km.plot(ax=axes[1], color="none", edgecolor=["red", "green", "blue"])
axes[0].set_title("5 km buffer")
axes[1].set_title("50 km buffer");

Note that both `.centroid` and `.buffer` return a `GeoSeries` object, even when the input is a `GeoDataFrame`:


In [ ]:
seine_buff_5km

In the common scenario when the original attributes of the input features need to be retained, you can replace the existing geometry with the new `GeoSeries` as in:


In [ ]:
seine_buff_5km = seine.copy()
seine_buff_5km['geometry'] = seine.buffer(5000)
seine_buff_5km

### Affine transformations {#sec-affine-transformations}

Affine transformation is any transformation that preserves lines and parallelism. However, angles or length are not necessarily preserved. Affine transformations include, among others, shifting (translation), scaling and rotation. Additionally, it is possible to use any combination of these. Affine transformations are an essential part of geocomputation. For example, shifting is needed for labels placement, scaling is used in non-contiguous area cartograms, and many affine transformations are applied when reprojecting or improving the geometry that was created based on a distorted or wrongly projected map. 

The `geopandas` package implements affine transformation, for objects of classes `GeoSeries` and `GeoDataFrame`. In both cases, the method is applied on the `GeoSeries` part, returning a new `GeoSeries` of transformed geometries. 

Affine transformations of `GeoSeries` can be done using the `.affine_transform` method, which is a wrapper around the `shapely.affinity.affine_transform` function. According to the [documentation](https://shapely.readthedocs.io/en/stable/manual.html#shapely.affinity.affine_transform), a 2D affine transformation requires a six-parameter list `[a,b,d,e,xoff,yoff]` which represents the following equations for transforming the coordinates:

$$
x' = a x + b y + x_\mathrm{off}
$$

$$
y' = d x + e y + y_\mathrm{off}
$$

There are also simplified `GeoSeries` [methods](https://geopandas.org/en/stable/docs/user_guide/geometric_manipulations.html#affine-transformations) for specific scenarios: 

* `GeoSeries.translate(xoff=0.0, yoff=0.0, zoff=0.0)`
* `GeoSeries.scale(xfact=1.0, yfact=1.0, zfact=1.0, origin='center')`
* `GeoSeries.rotate(angle, origin='center', use_radians=False)`
* `GeoSeries.skew(angle, origin='center', use_radians=False)`

For example, *shifting* only requires the $x_{off}$ and $y_{off}$, using `.translate`. The code below shifts the y-coordinates by 100,000 meters to the north, but leaves the x-coordinates untouched:


In [ ]:
nz_shift = nz.translate(0, 100000)

Scaling enlarges or shrinks objects by a factor. It can be applied either globally or locally. Global scaling increases or decreases all coordinates values in relation to the origin coordinates, while keeping all geometries topological relations intact. 

`Geopandas` implements local scaling using the `.scale` method. Local scaling treats geometries independently and requires points around which geometries are going to be scaled, e.g., centroids. In the example below, each geometry is shrunk by a factor of two around the centroids (middle panel in @fig-affine-transformations). To achieve that, we pass the `0.5` and `0.5` scaling factors (for x and y, respectively), and the `"centroid"` option for the point of origin. (Other than `"centroid"`, it is possible to use `"center"` for the bounding box center, or specific point coordinates.)


In [ ]:
nz_scale = nz.scale(0.5, 0.5, origin="centroid")

Rotating the geometries can be done using the `.rotate` method. When rotating, we need to specify the rotation angle (positive values imply clockwise rotation) and the `origin` points (using the same options as in `scale`). For example, the following expression rotates `nz` by 30 degrees counter-clockwise, around the geometry centroids:


In [ ]:
nz_rotate = nz.rotate(-30, origin="centroid")

@fig-affine-transformations shows the original layer `nz`, and the shifting, scaling and rotation results.


In [ ]:
#| label: fig-affine-transformations
#| fig-cap: 'Illustrations of affine transformations: shift, scale and rotate.'

fig, axes = plt.subplots(ncols=3, figsize=(8,4))
nz.plot(ax=axes[0], color="lightgrey", edgecolor="darkgrey")
nz_shift.plot(ax=axes[0], color="red", edgecolor="darkgrey")
nz.plot(ax=axes[1], color="lightgrey", edgecolor="darkgrey")
nz_scale.plot(ax=axes[1], color="red", edgecolor="darkgrey")
nz.plot(ax=axes[2], color="lightgrey", edgecolor="darkgrey")
nz_rotate.plot(ax=axes[2], color="red", edgecolor="darkgrey")
axes[0].set_title("Shift")
axes[1].set_title("Scale")
axes[2].set_title("Rotate");

### Clipping {#sec-clipping}

Spatial clipping is a form of spatial subsetting that involves changes to the geometry columns of at least some of the affected features.

Clipping can only apply to features more complex than points: lines, polygons and their 'multi' equivalents. To illustrate the concept we will start with a simple example: two overlapping circles with a center point one unit away from each other and a radius of one (@fig-overlapping-circles).


In [ ]:
#| label: fig-overlapping-circles
#| fig-cap: Overlapping circles.

x = shapely.geometry.Point((0, 0)).buffer(1)
y = shapely.geometry.Point((1, 0)).buffer(1)
shapely.geometry.GeometryCollection([x, y])

Imagine you want to select not one circle or the other, but the space covered by both x and y. This can be done using the `.intersection` method from `shapely`, illustrated using objects named `x` and `y` which represent the left- and right-hand circles (@fig-intersection).


In [ ]:
#| label: fig-intersection
#| fig-cap: Overlapping circles with a gray color indicating intersection between them.

x.intersection(y)

The next lines of code demonstrate how this works for the `.difference`, `.union`, and `.symmetric_difference` operators:


In [ ]:
x.difference(y)

In [ ]:
x.union(y)

In [ ]:
x.symmetric_difference(y)

### Subsetting and clipping

Clipping objects can change their geometry but it can also subset objects, returning only features that intersect (or partly intersect) with a clipping/subsetting object. 
To illustrate this point, we will subset points that cover the bounding box of the circles x and y in @fig-overlapping-circles. 
Some points will be inside just one circle, some will be inside both and some will be inside neither. 
The following code sections generates a simple random distribution of points within the extent of circles x and y, resulting in output illustrated in @fig-random-points. 
We do this in two steps. First, we figure out the bounds where random points are to be generated:


In [ ]:
bounds = x.union(y).bounds
bounds

Second, we use `np.random.uniform` to calculate `n` random x and y coordinates within the given bounds:


In [ ]:
np.random.seed(1)
n = 10  ## Number of points to generate
coords_x = np.random.uniform(bounds[0], bounds[2], n)
coords_y = np.random.uniform(bounds[1], bounds[3], n)
coords = list(zip(coords_x, coords_y))
coords

Third, we transform the list of coordinates into a `list` of `shapely` points:


In [ ]:
pnt = [shapely.geometry.Point(i) for i in coords]
pnt

and then to a `GeoSeries`:


In [ ]:
pnt = gpd.GeoSeries(pnt)
pnt

The result is shown in @fig-random-points:


In [ ]:
#| label: fig-random-points
#| fig-cap: Randomly distributed points within the bounding box enclosing circles x and y. The point that intersects with both objects x and y are highlighted.

base = pnt.plot(color='none', edgecolor='black')
gpd.GeoSeries([x]).plot(ax=base, color='none', edgecolor='darkgrey');
gpd.GeoSeries([y]).plot(ax=base, color='none', edgecolor='darkgrey');

Now, we get back to our question: how to subset the points to only return the point that intersects with both x and y? 
The code chunks below demonstrate three ways to achieve the same result. 
We can calculate a boolean `Series`, evaluating whether each point of `pnt` intersects with the intersection of x and y: 


In [ ]:
sel = pnt.intersects(x.intersection(y))
sel

then use it to subset `pnt` to get the result `pnt1`:


In [ ]:
pnt1 = pnt[sel]
pnt1

We can also find the intersection between the input points represented by `pnt`, using the intersection of `x` and `y` as the subsetting/clipping object. Since the second argument is an individual `shapely` geometry (`x.intersection(y)`), we get "pairwise" intersections of each `pnt` with it: 


In [ ]:
pnt2 = pnt.intersection(x.intersection(y))
pnt2

Empty geometries can be filtered out to retain the required subset, and to get `pnt2` which is identical to `pnt1`:


In [ ]:
pnt2 = pnt2[~pnt2.is_empty]  ## Subset non-empty geometries
pnt2

This second approach will return features that partly intersect with `x.intersection(y)` but with modified geometries for spatially extensive features that cross the border of the subsetting object. The results are identical, but the implementation differs substantially.

Although the example above is rather contrived and provided for educational rather than applied purposes, and we encourage the reader to reproduce the results to deepen your understanding for handling geographic vector objects in R, it raises an important question: which implementation to use? Generally, more concise implementations should be favored, meaning the first approach above. We will return to the question of choosing between different implementations of the same technique or algorithm in Chapter 11.

### Geometry unions {#sec-geometry-unions}

As we saw in @sec-vector-attribute-aggregation, spatial aggregation can silently dissolve the geometries of touching polygons in the same group. This is demonstrated in the code chunk below in which 49 `us_states` are aggregated into 4 regions using the `.dissolve` method:


In [ ]:
regions = us_states.dissolve(by='REGION', aggfunc='sum').reset_index()
regions

The result is shown in @fig-dissolve:


In [ ]:
#| label: fig-dissolve
#| fig-cap: 'Spatial aggregation on contiguous polygons, illustrated by aggregating the population of US states into regions, with population represented by color. Note the operation automatically dissolves boundaries between states.'

fig, axes = plt.subplots(ncols=2, figsize=(9, 2.5))
us_states.plot(ax=axes[0], edgecolor='black', column='total_pop_15', legend=True)
regions.plot(ax=axes[1], edgecolor='black', column='total_pop_15', legend=True)
axes[0].set_title('State')
axes[1].set_title('Region');

What is going on in terms of the geometries? Behind the scenes, `.dissolve` combines the geometries and dissolve the boundaries between them using the [`.unary_union`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.unary_union.html#geopandas.GeoSeries.unary_union) method per group. This is demonstrated in the code chunk below which creates a united western US using the standalone `unary_union` operation:


In [ ]:
us_west = us_states[us_states['REGION'] == 'West']
us_west_union = us_west['geometry'].unary_union

Note that the result is a `shapely` geometry, as the individual attributes are "lost" as part of dissolving. The result is shown in @fig-dissolve2.


In [ ]:
#| label: fig-dissolve2
#| fig-cap: Western US

us_west_union

To dissolve two (or more) groups of a `GeoDataFrame` into one geometry, we can either use a combined condition:


In [ ]:
sel = (us_states['REGION'] == 'West') | (us_states['NAME'] == 'Texas')
texas_union = us_states[sel]
texas_union = texas_union['geometry'].unary_union

or concatenate the two separate subsets:


In [ ]:
us_west = us_states[us_states['REGION'] == 'West']
texas = us_states[us_states['NAME'] == 'Texas']
texas_union = pd.concat([us_west, texas]).unary_union

and then dissove using `.unary_union`. The result is identical in both cases, shown in @fig-dissolve3.


In [ ]:
#| label: fig-dissolve3
#| fig-cap: Western US and Texas

texas_union

### Type transformations {#sec-type-transformations}

Transformation of geometries, from one type to another, also known as "geometry casting", is often required to facilitate spatial analysis. 
The `shapely` package can be used for geometry casting. 
The exact expression(s) depend on the specific transformation we are interested in. 
In general, you need to figure out the required input of the respective construstor function according to the "destination" geometry (e.g., `shapely.geometry.LineString`, etc.), then reshape the input of the "source" geometry into the right form to be passed to that function.

Let's create a `"MultiPoint"` to illustrate how geometry casting works on `shapely` geometry objects:


In [ ]:
multipoint = shapely.geometry.MultiPoint([(1,1), (3,3), (5,1)])
multipoint

A `"LineString"` can be created using `shapely.geometry.LineString` from a `list` of points. Consequently, a `"MultiPoint"` can be converted to a `"LineString"` by extracting the individual points into a `list`, then passing them to `shapely.geometry.LineString`:


In [ ]:
linestring = shapely.geometry.LineString(list(multipoint.geoms))
linestring

A `"Polygon"` can also be created using function `shapely.geometry.Polygon`, which acceps a sequence of point coordinates. In principle, the last coordinate must be equal to the first, in order to form a closed shape. However, `shapely.geometry.Polygon` is able to complete the last coordinate automatically. Therefore:


In [ ]:
polygon = shapely.geometry.Polygon([[p.x, p.y] for p in multipoint.geoms])
polygon

The source `"MultiPoint"` geometry, and the derived `"LineString"` and `"Polygon"` geometries are shown in @fig-casting1. Note that we convert the `shapely` geometries to `GeoSeries` for easier multi-panel plotting:


In [ ]:
#| label: fig-casting1
#| fig-cap: Examples of linestring and polygon casted from a multipoint geometry.

fig, axes = plt.subplots(ncols=3, figsize=(8,4))
gpd.GeoSeries(multipoint).plot(ax=axes[0])
gpd.GeoSeries(linestring).plot(ax=axes[1])
gpd.GeoSeries(polygon).plot(ax=axes[2])
axes[0].set_title("MultiPoint")
axes[1].set_title("LineString")
axes[2].set_title("Polygon");

Conversion from multipoint to linestring is a common operation that creates a line object from ordered point observations, such as GPS measurements or geotagged media. This allows spatial operations such as the length of the path traveled. Conversion from multipoint or linestring to polygon is often used to calculate an area, for example from the set of GPS measurements taken around a lake or from the corners of a building lot.

Our `"LineString"` geometry can be converted bact to a `"MultiPoint"` geometry by passing its coordinates directly to `shapely.geometry.MultiPoint`:


In [ ]:
# 'LineString' -> 'MultiPoint'
shapely.geometry.MultiPoint(linestring.coords)

The `"Polygon"` (exterior) coordinates can be passed to `shapely.geometry.MultiPoint` as well:


In [ ]:
# 'Polygon' -> 'MultiPoint'
shapely.geometry.MultiPoint(polygon.exterior.coords)

...

## Geometric operations on raster data {#sec-geo-ras}

### Geometric intersections

...

### Extent and origin

...

### Aggregation and disaggregation {#sec-raster-agg-disagg}

Raster datasets vary based on their resolution, from high resolution datasets that enable individual trees to be seen, to low resolution datasets covering the whole Earth.
Raster datasets can be transformed to either decrease (aggregate) or increase (disaggregate) their resolution for a number of reasons.
Aggregation can reduce computational resource requirements of raster storage and subsequent steps, disaggregation can be used to match other datasets or to add detail.
As an example, we here change the spatial resolution of `dem.tif` by a factor of 5 (Figure @fig-raster-aggregate). 

Raster aggregation is, in fact, a special case of raster resampling (see @sec-raster-resampling), where the target raster grid is aligned with the original raster, only with coarser pixels. Raster resampling, is the general case where the new grid is not necessarily an aggregation of the original one, but any other case as well (such as a rotated and/or shifted one, etc.). 

To aggregate a raster using `rasterio`, we go through [two steps](https://rasterio.readthedocs.io/en/stable/topics/resampling.html):

* Reading the raster values (using `.read`) into an `out_shape` that is different from the original `.shape`
* Updating the `transform` according to the `out_shape`

Let's demonstrate. First, we create a file connection to `dem.tif`, using `rasterio.open`:


In [ ]:
src = rasterio.open('data/dem.tif')

Note the shape of the raster, it has 117 rows and 117 columns:


In [ ]:
src.read(1).shape

Also note the `transform`, which tells us that the raster resolution is 30.85 $m$:


In [ ]:
src.transform

Now, instead of reading the raster values the usual way, as in `src.read(1)`, we can specify `out_shape` to read the values into a different shape. 
Here, we calculate a new shape which is downscaled by a factor of `5`, i.e., the number of rows and columns is multiplied by `0.2`. 
We must truncate any "partial" rows and columns, e.g., using `int`. 
Each new pixel is now obtained, or "resampled", from $\sim 5 \times 5 = \sim 25$ "old" raster values. 
We can choose the resampling method through the `resampling` parameter. 
Here we use `rasterio.enums.Resampling.average`, i.e., the new "large" pixel value is the average of all coinciding small pixels, which makes sense for our elevation data in `dem.tif`:


In [ ]:
factor = 0.2
r = src.read(1,
    out_shape=(
        int(src.height * factor),
        int(src.width * factor)
        ),
    resampling=rasterio.enums.Resampling.average
)

The resulting array `r` has a smaller `.shape`, as shown below:


In [ ]:
r.shape

Other useful [options](https://rasterio.readthedocs.io/en/stable/api/rasterio.enums.html#rasterio.enums.Resampling) include:

* `rasterio.enums.Resampling.nearest`---Nearest neighbor resampling
* `rasterio.enums.Resampling.bilinear`---Bilinear resampling
* `rasterio.enums.Resampling.cubic`---Cubic resampling
* `rasterio.enums.Resampling.lanczos`---Lanczos windowed resampling
* `rasterio.enums.Resampling.mode`---Mode resampling (most common value)
* `rasterio.enums.Resampling.min`---Minimum resampling
* `rasterio.enums.Resampling.max`---Maximum resampling
* `rasterio.enums.Resampling.med`---Median resampling
* `rasterio.enums.Resampling.sum`---Median resampling

See below (@sec-raster-resampling) for an explanation of these methods.

The second step is to update the `transform`, taking into account the change in raster shape, as follows:


In [ ]:
new_transform = src.transform * src.transform.scale(
    (src.width / r.shape[0]),
    (src.height / r.shape[1])
)
new_transform

The original raster and the aggregated one, are shown in @fig-raster-aggregate:


In [ ]:
#| label: fig-raster-aggregate
#| fig-cap: 'Original raster (left), and aggregated raster (right).'

fig, axes = plt.subplots(ncols=2, figsize=(8,4))
rasterio.plot.show(src, ax=axes[0])
rasterio.plot.show(r, transform=new_transform, ax=axes[1])
axes[0].set_title('Original')
axes[1].set_title('Aggregated (average)');

In case we need to export the new raster, we need to update the metadata: 


In [ ]:
dst_kwargs = src.meta.copy()
dst_kwargs.update({
    'transform': new_transform,
    'width': r.shape[0],
    'height': r.shape[1],
})
dst_kwargs

Then create a new file in writing mode, and write the values in `r` into that file (see @sec-data-output-raster): 


In [ ]:
dst = rasterio.open('output/dem_agg5.tif', 'w', **dst_kwargs)
dst.write(r, 1)
dst.close()

The opposite operastion, disaggregation, is when we increase the resolution of raster objects. Either of the supported resampling methods (see above) can be used. However, since we are not actually summarizing information but transferring the value of a large pixel into multiple small pixels, it makes sense to use either:

* Nearest neighbor resampling (`rasterio.enums.Resampling.nearest`), when want to keep the original values as-is, since when modifying them would be incorrect (such as in categorical rasters)
* Smooting techniques, such as Bilinear resampling (`rasterio.enums.Resampling.bilinear`), when we would like the smaller pixels to reflect gradual change between the original values, e.g., when the disaggregated raster is used for visulalization purposes

To disaggregate a raster, we go through the same workflow as for aggregation, only using a different factor, such as `factor=5` instead of `factor=0.2`, i.e., *increasing* the number of raster pixels instead of decreasing. In this example, we use bilinear interpolation to get a smoothed high-resolution raster:


In [ ]:
factor = 5
r2 = src.read(1,
    out_shape=(
        int(src.height * factor),
        int(src.width * factor)
        ),
    resampling=rasterio.enums.Resampling.bilinear
)

Here is the size of the disaggregated raster:


In [ ]:
r2.shape

And here is the same expression as shown for aggregation, to calculate the new transform:


In [ ]:
new_transform2 = src.transform * src.transform.scale(
    (src.width / r2.shape[0]),
    (src.height / r2.shape[1])
)
new_transform2

A zoom-in on the top-left corner of the original raster and the disaggregated one, are shown in @fig-raster-disaggregate:


In [ ]:
#| label: fig-raster-disaggregate
#| fig-cap: 'Original raster (left), and disaggregated raster (right). The same top-left corner extent of both rasters is shown, to zoom-in and demonstrate the effect of bilinear interpolation.'

fig, axes = plt.subplots(ncols=2, figsize=(8,4))
rasterio.plot.show(src.read(1)[:5, :5], transform=src.transform, ax=axes[0])
rasterio.plot.show(r2[:25, :25], transform=new_transform2, ax=axes[1])
axes[0].set_title('Original')
axes[1].set_title('Disaggregated (bilinear)');

Code to export the disaggregated raster would be identical to the one used above for the aggregated raster, so omit it to save space.

### Resampling {#sec-raster-resampling}

The above methods of aggregation and disaggregation are only suitable when we want to change the resolution of our raster by the aggregation/disaggregation factor. 
However, what to do when we have two or more rasters with different resolutions and origins? 
This is the role of resampling---a process of computing values for new pixel locations. 
In short, this process takes the values of our original raster and recalculates new values for a target raster with custom resolution and origin (@fig-raster-resample).

There are several methods for estimating values for a raster with different resolutions/origins (@fig-raster-resample). The main resampling methods include:

* Nearest neighbor: assigns the value of the nearest cell of the original raster to the cell of the target one. This is a fast simple technique that is usually suitable for resampling categorical rasters.
* Bilinear interpolation: assigns a weighted average of the four nearest cells from the original raster to the cell of the target one. This is the fastest method that is appropriate for continuous rasters.
* Cubic interpolation: uses values of the 16 nearest cells of the original raster to determine the output cell value, applying third-order polynomial functions. Used for continuous rasters and results in a smoother surface compared to bilinear interpolation, but is computationally more demanding.
* Cubic spline interpolation: also uses values of the 16 nearest cells of the original raster to determine the output cell value, but applies cubic splines (piecewise third-order polynomial functions). Used for continuous rasters.
* Lanczos windowed sinc resampling: uses values of the 36 nearest cells of the original raster to determine the output cell value. Used for continuous rasters.
* Additionally, we can use straightforward summary methods, taking into account all pixels that coincide with the target pixel, such as average (@fig-raster-aggregate), minimum, maximum (@fig-raster-resample), median, mode, and sum.

The above explanation highlights that only nearest neighbor resampling is suitable for categorical rasters, while all remaining methods can be used (with different outcomes) for continuous rasters. 

With `rasterio`, resampling can be done using function `rasterio.warp.reproject`. 
Note, again, that raster reprojection is not fundamentally different from resampling---the difference is just whether the target grid is in the same CRS as the origin (resampling) or in a different CRS (reprojection). 
In other words, reprojection is *resampling* into a grid that is in a different CRS. We will demonstrate reprojection using `rasterio.warp.reproject` later on (@sec-reprojecting-raster-geometries). 

The information required for `rasterio.warp.reproject`, whether we are resampling or reprojecting, is:

* The source and target *CRS*. These may be identical, when resampling, or different, when reprojecting.
* The source and target *transform*

Also, `rasterio.warp.reproject` works with file connections, so it requires a connection to an output file in write (`'w'`) mode. This makes the function efficient for large rasters.

The target and destination CRS are straightforward to specify, depending on our choice. The source transform is also available, e.g., from the source file connection. 
The only complicated part is to figure out the *destination transform*. 
When resampling, the transform is typically derived from a *template* raster, such as an existing raster file that we would like our origin raster to match, or a numeric specification of our target grid (see below). 
Otherwise, when the exact grid is not of importance, we can simply aggregate or disaggregate our raster as shown above (@sec-raster-agg-disagg).
(Note that when reprojecting, the target transform is not straightforward to figure out, therefore we use the `rasterio.warp.calculate_default_transform` function to calculate it, as will be shown in @sec-reprojecting-raster-geometries.)

Let's demonstrate resampling into a destination grid which is specified through numeric contraints, such as the extent and resolution:


In [ ]:
xmin = 794650
xmax = 798250
ymin = 8931750 
ymax = 8935350
res = 300

The transform is a function of the origin and resolution, and can be created using the `rasterio.transform.from_origin` function as follows:


In [ ]:
dst_transform = rasterio.transform.from_origin(west=xmin, north=ymax, xsize=res, ysize=res)
dst_transform

Again, note that in case we needed to resample into a grid specified by an existing "template" raster, we could skip this step and simply use read the transform from that file, as in `rasterio.open('template.tif').transform`.

Now we move on to creating the destination file connection. 
For that, we also have to know the raster dimensions. 
These can be derived from the extent and the resolution, as follows:


In [ ]:
width = int((xmax - xmin) / res)
height = int((ymax - ymin) / res)
width, height

Now we create the destination file connection. 
We are using the same metadata as the source file, except for the dimensions and the transform, which are going to be different and reflecting the resampling process:


In [ ]:
dst_kwargs = src.meta.copy()
dst_kwargs.update({
    'transform': dst_transform,
    'width': width,
    'height': height
})
dst = rasterio.open('output/dem_resample_nearest.tif', 'w', **dst_kwargs)

Here is how we reproject, using function `rasterio.warp.reproject`. The resampling method being used here is nearest neighbor resampling:


In [ ]:
rasterio.warp.reproject(
    source=rasterio.band(src, 1),
    destination=rasterio.band(dst, 1),
    src_transform=src.transform,
    src_crs=src.crs,
    dst_transform=dst_transform,
    dst_crs=src.crs,
    resampling=rasterio.enums.Resampling.nearest
)

In the end, we close the file:


In [ ]:
dst.close()

Here is another code section to demontrate another resampling method, the maximum resampling, i.e., every new pixel gets the maximum value of all the original pixels it coincides with. Note that the transform is identical (@fig-raster-resample), so we do not need to calculate it again:


In [ ]:
dst = rasterio.open('output/dem_resample_maximum.tif', 'w', **dst_kwargs)
rasterio.warp.reproject(
    source=rasterio.band(src, 1),
    destination=rasterio.band(dst, 1),
    src_transform=src.transform,
    src_crs=src.crs,
    dst_transform=dst_transform,
    dst_crs=src.crs,
    resampling=rasterio.enums.Resampling.max
)
dst.close()

The original raster `dem.tif`, and the two resampling results `dem_resample_nearest.tif` and `dem_resample_maximum.tif`, are shown in @fig-raster-resample:


In [ ]:
#| label: fig-raster-resample
#| fig-cap: 'Visual comparison of the original raster and two different resampling methods: nearest neighbor and maximum resampling.'

fig, axes = plt.subplots(ncols=3, figsize=(8,4))
rasterio.plot.show(src, ax=axes[0])
rasterio.plot.show(rasterio.open('output/dem_resample_nearest.tif'), ax=axes[1])
rasterio.plot.show(rasterio.open('output/dem_resample_maximum.tif'), ax=axes[2])
axes[0].set_title('Original')
axes[1].set_title('Nearest neighbor')
axes[2].set_title('Maximum');

## Exercises
